In [1]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from random import randint
import  random
import os
# os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from cmath import nan
import urllib
import urllib.request
import re
import time


#### Get sitemap for fighters

In [2]:
url = 'https://www.tapology.com/fightcenter/fighters/sitemap.xml'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
r = requests.get(url, headers = header)
r.text

# turn into df
df = pd.DataFrame(r.text.split('\n'))
df.columns = ['url']
df = df[df['url'].str.contains('fighter')]
df

,url
3,<loc>https://www.tapology.com/fightcenter/...
9,<loc>https://www.tapology.com/fightcenter/...
15,<loc>https://www.tapology.com/fightcenter/...
21,<loc>https://www.tapology.com/fightcenter/...
27,<loc>https://www.tapology.com/fightcenter/...
...,...
26907,<loc>https://www.tapology.com/fightcenter/...
26913,<loc>https://www.tapology.com/fightcenter/...
26919,<loc>https://www.tapology.com/fightcenter/...
26925,<loc>https://www.tapology.com/fightcenter/...


Get fighter first and last names. 

Some contain middle names, and most also contain nicknames. 
It typically goes: First-Last-The-Nickname

In [3]:
df['whole_name'] = df['url'].str.split('/').str[-2]
df

,url,whole_name
3,<loc>https://www.tapology.com/fightcenter/...,dan-henderson-hollywood<
9,<loc>https://www.tapology.com/fightcenter/...,rich-franklin-ace<
15,<loc>https://www.tapology.com/fightcenter/...,mauricio-rua-shogun<
21,<loc>https://www.tapology.com/fightcenter/...,mark-coleman-the-hammer<
27,<loc>https://www.tapology.com/fightcenter/...,alan-belcher-the-talent<
...,...,...
26907,<loc>https://www.tapology.com/fightcenter/...,4993-jennifer-howe<
26913,<loc>https://www.tapology.com/fightcenter/...,4994-anthony-marti<
26919,<loc>https://www.tapology.com/fightcenter/...,4995-ana-carolina<
26925,<loc>https://www.tapology.com/fightcenter/...,4996-tommy-hayden<


In [4]:
# if whole_name has numbers, remove them
df['whole_name'] = df['whole_name'].str.replace(r'\d+', '')
# if first character is a number, remove number
df['whole_name'] = df['whole_name'].str.replace(r'^\d+','')
# if first character is a space, remove space
df['whole_name'] = df['whole_name'].str.replace(r'^\s+','')
df

,url,whole_name
3,<loc>https://www.tapology.com/fightcenter/...,dan-henderson-hollywood<
9,<loc>https://www.tapology.com/fightcenter/...,rich-franklin-ace<
15,<loc>https://www.tapology.com/fightcenter/...,mauricio-rua-shogun<
21,<loc>https://www.tapology.com/fightcenter/...,mark-coleman-the-hammer<
27,<loc>https://www.tapology.com/fightcenter/...,alan-belcher-the-talent<
...,...,...
26907,<loc>https://www.tapology.com/fightcenter/...,-jennifer-howe<
26913,<loc>https://www.tapology.com/fightcenter/...,-anthony-marti<
26919,<loc>https://www.tapology.com/fightcenter/...,-ana-carolina<
26925,<loc>https://www.tapology.com/fightcenter/...,-tommy-hayden<


In [5]:
# if whole_name starts with -, remove only first character
df['whole_name'] = df['whole_name'].apply(lambda x: x[1:] if x.startswith('-') else x)
df

,url,whole_name
3,<loc>https://www.tapology.com/fightcenter/...,dan-henderson-hollywood<
9,<loc>https://www.tapology.com/fightcenter/...,rich-franklin-ace<
15,<loc>https://www.tapology.com/fightcenter/...,mauricio-rua-shogun<
21,<loc>https://www.tapology.com/fightcenter/...,mark-coleman-the-hammer<
27,<loc>https://www.tapology.com/fightcenter/...,alan-belcher-the-talent<
...,...,...
26907,<loc>https://www.tapology.com/fightcenter/...,jennifer-howe<
26913,<loc>https://www.tapology.com/fightcenter/...,anthony-marti<
26919,<loc>https://www.tapology.com/fightcenter/...,ana-carolina<
26925,<loc>https://www.tapology.com/fightcenter/...,tommy-hayden<


In [6]:
# delete all '<'
df['whole_name'] = df['whole_name'].str.replace('<', '')
# replace - with space
df['whole_name'] = df['whole_name'].str.replace('-', ' ')
# add column to count words in whole_name
df['word_count'] = df['whole_name'].str.split().str.len()
df

,url,whole_name,word_count
3,<loc>https://www.tapology.com/fightcenter/...,dan henderson hollywood,3
9,<loc>https://www.tapology.com/fightcenter/...,rich franklin ace,3
15,<loc>https://www.tapology.com/fightcenter/...,mauricio rua shogun,3
21,<loc>https://www.tapology.com/fightcenter/...,mark coleman the hammer,4
27,<loc>https://www.tapology.com/fightcenter/...,alan belcher the talent,4
...,...,...,...
26907,<loc>https://www.tapology.com/fightcenter/...,jennifer howe,2
26913,<loc>https://www.tapology.com/fightcenter/...,anthony marti,2
26919,<loc>https://www.tapology.com/fightcenter/...,ana carolina,2
26925,<loc>https://www.tapology.com/fightcenter/...,tommy hayden,2


In [7]:
def extract_nickname(row):
    # check the word_count column to see if there is a nickname
    if row['word_count'] > 2:
        # fighter name is first two words in whole_name
        row['Fighter Name'] = ' '.join(row['whole_name'].split(' ')[:2])
        # nickname is the last part
        row['Nickname'] = ' '.join(row['whole_name'].split(' ')[-2:])
    else:
        row['Fighter Name'] = row['whole_name']
        row['Nickname'] = np.nan
        
    return row

# Apply the function
df = df.apply(extract_nickname, axis=1)
df.head(50)

,url,whole_name,word_count,Fighter Name,Nickname
3,<loc>https://www.tapology.com/fightcenter/...,dan henderson hollywood,3,dan henderson,henderson hollywood
9,<loc>https://www.tapology.com/fightcenter/...,rich franklin ace,3,rich franklin,franklin ace
15,<loc>https://www.tapology.com/fightcenter/...,mauricio rua shogun,3,mauricio rua,rua shogun
21,<loc>https://www.tapology.com/fightcenter/...,mark coleman the hammer,4,mark coleman,the hammer
27,<loc>https://www.tapology.com/fightcenter/...,alan belcher the talent,4,alan belcher,the talent
33,<loc>https://www.tapology.com/fightcenter/...,denis kang super korean,4,denis kang,super korean
39,<loc>https://www.tapology.com/fightcenter/...,chris lytle lights out,4,chris lytle,lights out
45,<loc>https://www.tapology.com/fightcenter/...,marcus davis the irish hand grenade,6,marcus davis,hand grenade
51,<loc>https://www.tapology.com/fightcenter/...,andrei arlovski the pitbull,4,andrei arlovski,the pitbull
57,<loc>https://www.tapology.com/fightcenter/...,fedor emelianenko the last emperor,5,fedor emelianenko,last emperor


In [8]:
df['sherdog_urls'] = df['url'].str.replace('<loc>', "").str.replace('<url>', '').str.strip()
df

,url,whole_name,word_count,Fighter Name,Nickname,sherdog_urls
3,<loc>https://www.tapology.com/fightcenter/...,dan henderson hollywood,3,dan henderson,henderson hollywood,https://www.tapology.com/fightcenter/fighters/...
9,<loc>https://www.tapology.com/fightcenter/...,rich franklin ace,3,rich franklin,franklin ace,https://www.tapology.com/fightcenter/fighters/...
15,<loc>https://www.tapology.com/fightcenter/...,mauricio rua shogun,3,mauricio rua,rua shogun,https://www.tapology.com/fightcenter/fighters/...
21,<loc>https://www.tapology.com/fightcenter/...,mark coleman the hammer,4,mark coleman,the hammer,https://www.tapology.com/fightcenter/fighters/...
27,<loc>https://www.tapology.com/fightcenter/...,alan belcher the talent,4,alan belcher,the talent,https://www.tapology.com/fightcenter/fighters/...
...,...,...,...,...,...,...
26907,<loc>https://www.tapology.com/fightcenter/...,jennifer howe,2,jennifer howe,NaN,https://www.tapology.com/fightcenter/fighters/...
26913,<loc>https://www.tapology.com/fightcenter/...,anthony marti,2,anthony marti,NaN,https://www.tapology.com/fightcenter/fighters/...
26919,<loc>https://www.tapology.com/fightcenter/...,ana carolina,2,ana carolina,NaN,https://www.tapology.com/fightcenter/fighters/...
26925,<loc>https://www.tapology.com/fightcenter/...,tommy hayden,2,tommy hayden,NaN,https://www.tapology.com/fightcenter/fighters/...


In [9]:
df= df.drop(['url'], axis = 1)
df

,whole_name,word_count,Fighter Name,Nickname,sherdog_urls
3,dan henderson hollywood,3,dan henderson,henderson hollywood,https://www.tapology.com/fightcenter/fighters/...
9,rich franklin ace,3,rich franklin,franklin ace,https://www.tapology.com/fightcenter/fighters/...
15,mauricio rua shogun,3,mauricio rua,rua shogun,https://www.tapology.com/fightcenter/fighters/...
21,mark coleman the hammer,4,mark coleman,the hammer,https://www.tapology.com/fightcenter/fighters/...
27,alan belcher the talent,4,alan belcher,the talent,https://www.tapology.com/fightcenter/fighters/...
...,...,...,...,...,...
26907,jennifer howe,2,jennifer howe,NaN,https://www.tapology.com/fightcenter/fighters/...
26913,anthony marti,2,anthony marti,NaN,https://www.tapology.com/fightcenter/fighters/...
26919,ana carolina,2,ana carolina,NaN,https://www.tapology.com/fightcenter/fighters/...
26925,tommy hayden,2,tommy hayden,NaN,https://www.tapology.com/fightcenter/fighters/...


In [11]:
os.chdir('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')

In [12]:
os.getcwd()

'/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2'

In [13]:
df.to_csv('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/data/final/aggregates/Sherdog_Fighter_URLs.csv', index = False)

## Scrape One

In [14]:
# get alexander volkanovski
# search for alexander volkanovski whole_name
df[df['whole_name'].str.contains('volkanovski')]

,whole_name,word_count,Fighter Name,Nickname,sherdog_urls


In [24]:
url = 'https://www.tapology.com/fightcenter/fighters/41705-alex-volkanski'

# get page with selenium
driver = webdriver.Chrome('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/chromedriver')
driver.get(url)

In [31]:
# scrape with requests, using a header
header = {
                    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
                    "X-Requested-With": "XMLHttpRequest"
    }

# scrape with requests
r = requests.get(url, headers = header)
soup = BeautifulSoup(r.text, 'html.parser')

# get Fighter Details
fighter_details = soup.find_all('div', {'class': 'details details_two_columns'})
fighter_details_text = fighter_details[0].text
# split on \n
fighter_details_text = fighter_details_text.split('\n')
# remove empty strings
fighter_details_text = list(filter(None, fighter_details_text))
# replace | with new line
fighter_details_text = [x.replace('|', '\n') for x in fighter_details_text]
# replace any '\n ' with ''
fighter_details_text = [x.replace('\n ', '') for x in fighter_details_text]
# replace any '\n' with ''
fighter_details_text


['Name:',
 'Alexander Volkanovski',
 'Pro MMA Record:',
 '25-2-0 (Win-Loss-Draw)',
 'Nickname:',
 'The Great',
 'Current Streak:',
 '1 Loss',
 'Age:',
 '1988-09-29',
 'Date of Birth:',
 '1988.09.29',
 'Last Fight:',
 'February 11, 2023',
 'in',
 'UFC',
 'Weight Class:',
 'Lightweight',
 'Last Weigh-In:',
 '154.5 lbs',
 'Affiliation:',
 'N/A',
 'Height:',
 '5\'6" (168cm)',
 'Reach:',
 '71.5" (182cm)',
 'Career Disclosed Earnings:',
 '$575,000 USD',
 'Born:',
 'Australia',
 'Fighting out of:',
 'Windang, New South Wales',
 'Head Coach:',
 'Joe Lopez',
 'Other Coaches:',
 'N/A',
 'Fighter Links:',
 'Personal Links:']

In [32]:
# get fighter record stats with ul class='fighterRecordStats'
fighter_record_stats = soup.find_all('ul', {'class': 'fighterRecordStats'})
fighter_record_stats_text = fighter_record_stats[0].text
# split on \n
fighter_record_stats_text = fighter_record_stats_text.split('\n')
# remove empty strings
fighter_record_stats_text = list(filter(None, fighter_record_stats_text))
fighter_record_stats_text

['KO/TKO',
 '12 wins,',
 '1 loss',
 'W',
 'L',
 '48%',
 'of wins',
 '50%',
 'of losses',
 'Submission',
 '3 wins,',
 '0 losses',
 'W',
 'L',
 '12%',
 'of wins',
 'Decision',
 '10 wins,',
 '1 loss',
 'W',
 'L',
 '40%',
 'of wins',
 '50%',
 'of losses']

In [49]:
# get mma record by promotion using ul class = 'fighterPromotions'
fighter_promotions = soup.find_all('section', {'class': 'fighterPromotions'})
fighter_promotions_text = fighter_promotions[0].text
# split on \n
fighter_promotions_text = fighter_promotions_text.split('\n')
# remove empty strings
fighter_promotions_text = list(filter(None, fighter_promotions_text))
fighter_promotions_text
# get all img ALT tags within fighter_promotions
fighter_promotions_img_alt = fighter_promotions[0].find_all('img', alt=True)
fighter_promotions_img_alt = [x['alt'] for x in fighter_promotions_img_alt]
fighter_promotions_img_alt



IndexError: list index out of range

In [56]:
# scrape fighter fight results (class="fighterFightResults")
fighter_fight_results = soup.find_all('section', {'class': 'fighterFightResults'})
# count number of li tags
len(fighter_fight_results[0].find_all('li'))

# each li tag is a fight. each fight contains oppenent div, summary div, date div, and more div. 
# grab all divs within each li tag
fighter_fight_results_li = fighter_fight_results[0].find_all('li')
# 

[<div class="result" data-result="upcoming">
 <div class="opponent">
 <div class="name">
 <a href="/fightcenter/fighters/70995-yair-rodriguez" title="Yair Rodriguez Fighter Page">Yair Rodriguez</a>
 </div>
 <div class="record">
 <span title="Fighter Record Before Fight">25-2-0</span>
 <span title="Opponent Record Before Fight">15-3-0</span>
 </div>
 </div>
 <div class="summary">
 <div class="lead">
 <a href="/fightcenter/bouts/750444-ufc-290-alexander-the-great-volkanovski-vs-yair-el-pantera-rodriguez" title="Bout Page">Confirmed Upcoming Bout</a>
 </div>
 <div class="notes">
 <a href="/fightcenter/events/98868-ufc-290" title="Event Page">UFC 290: Volkanovski vs. Rodriguez</a>
 </div>
 </div>
 <div class="date">
 2023.07.08
 </div>
 <div class="more" id="fighterRecordMore">
 <i class="toggle" data-action="click-&gt;fighter-bout#toggleDetail" data-fighter-bout-bout-id-param="750444" title="Show Details">add_circle</i>
 </div>
 </div>,
 <div class="opponent">
 <div class="name">
 <a href

## Scrape All